# Introduction

The Variational Quantum Simulation theory allows to describe the time evolution of a parameterized state $ |\Psi(t)\rangle = |\Psi (\{\lambda_i(t)\}_{i=1}^N) \rangle$ through the ordinary differential equation (ODE),
$$ \frac{d\lambda_i}{dt} = \left[M^{-1}\right]_{ik} V_k $$
where the matrix $M$ and the vector $V$ can be computed by means of the Hamiltonian of the system, and a quantum circuit, as presented on Sec. 2.

In this spirit, we implemented a python library which can solve this ODE by computing the relevant quantities on a real quantum device provided by IBM through Qiskit, and perform a fully working simulation for a quantum system of interest. 

To accomplish this goal, the library which we named `VarQuS`, is organized as follows
```
 varqus
   └─── ode.py
   └─── variational_simulation.py
   └─── analytic.py
   └─── integrators.py
   └─── states.py
   └─── utils.py
```

## Define a problem

In the following, we will briefly describe each part of the library, but in order to give exemples as we go through it, we will begin by defining a system to solve

In [1]:
import numpy as np

# Circuit information
circuit_coefs     = [[-0.5j], [-0.5j, -0.5j]]
circuit_operators = [["ZZ"],  [ "XI",  "IX"]]

# System information
hamiltonian_coefs     = [-1j, -0.5j, -0.5j]
hamiltonian_operators = ["ZZ", "XI",  "IX"]

# Initial state and parameters
initial_state  = np.ones(4, dtype=complex)/2
initial_params = np.array([1.0, 1.0])

Note that a time-dependent Hamiltonian could also be provided, for example, with coefficients:
```python
hamiltonian_coefs = lambda t: [0.5*np.sin(t), 2.0*np.cos(t)]
```

## ODE
The first file, named `ode.py` provides an interface for the final user, a function called `define_vqs_ode`, which takes the information of the circuit and the Hamiltonian of the system, and defines another function, representing the ODE to solve, which can be evaluated for a group of parameters, $\lambda_i(t_n)$ and its corresponding time, $t_n$, to obtain $d\lambda_i/dt$.

In [2]:
from varqus.ode import define_vqs_ode

vqs_analytic  = define_vqs_ode(circuit_operators, hamiltonian_operators, circuit_coefs, hamiltonian_coefs, initial_state, backend='analytic')
vqs_simulated = define_vqs_ode(circuit_operators, hamiltonian_operators, circuit_coefs, hamiltonian_coefs, initial_state, shots=2**13)

Note that this function takes a keyword `backend`, which accepts a qiskit backend and defaults to the Qiskit Aer simulator, but can be changed to run on a real quantum device. If `backend='analytic'` is passed, then the system is solved exactly by means of linear algebra operations instead of running in a backend. For any backend different to `'analytic'`, the number of shots must be provided with the `shots` keyword.

Similarly, `ode.py` also provides a function called `define_schrodinger_ode` which takes the information of the system and defines an ODE, corresponding to the time-dependent Schrodinger equation.

In [3]:
from varqus.ode import define_schrodinger_ode

schrodinger_ode = define_schrodinger_ode(hamiltonian_operators, hamiltonian_coefs)

## Variational simulation

The file `variational_simulation.py` provides all the methods required to calculate the matrix `M` and the vector `V` in a quantum circuit. This tools are managed internally by the ODE defined, and therefore, they are not of interest for the final user.

## Analytic

The file `analytic.py` provides the methods to calculate `M` and `V` analytically. That is, the Variational Quantum Simulation theory is used and the equations are directly solved, instead of running them in a backend. Just like `variational_simulation.py`, the methods defined on this file are not of interest for the final user.

## Integrators

The file `integrators.py` provide integrators which can take the ODEs previously defined and solve it for some time span. As of now, two integrators are implemented: the first order `euler` and the forth order Runge-Kutta `rk4`. Both integrators accept the same arguments.

In [7]:
# Set time discretization
dt = 0.01 # time step
Nt = 10   # number of time steps

from varqus.integrators import euler, rk4

# Here we solve the previously posed ODE's
vqs_analytic_evolved  = euler(vqs_analytic,  initial_params, dt, Nt)
vqs_simulated_evolved = euler(vqs_simulated, initial_params, dt, Nt)

# We can solve the Schrodinger ODE too (Remember the Schrodinger equation acts over the state, not the parameters)
schrodinger_evolved = euler(schrodinger_ode, initial_state, dt, Nt)

## States
The file `states.py` provides some basic tools to work on states. They are useful to the end user.

In [15]:
from varqus.states import state_infidelity, state_from_parameters

# With 'state_from_parameters' we can recover the parameters 
simulated_states = [state_from_parameters(params, circuit_operators, circuit_coefs, initial_state) for params in vqs_simulated_evolved]
analytic_states  = [state_from_parameters(params, circuit_operators, circuit_coefs, initial_state) for params in vqs_analytic_evolved]

# State infidelity is a distance between states. If both states are the same, their infidelity will be zero.
vqs_infidelities = [state_infidelity(simulated_states[t], schrodinger_evolved[t]) for t in range(Nt)]

## Utils

The file `utils.py` provides utilities needed by the rest of the library, for example, to parse the matrix form of the operator from their name. This 